In [1]:
from classify import data_pipeline, eco_selector
from resources.get_data import get_current_data
from resources.setup import get_setup
import pandas as pd

In [2]:
def validator(df):
    df = df.drop_duplicates()
    df = df[~df['text'].isna()]
    df['text'] = df['text'].astype(str)
    if 'vectorized' in df.columns.values:
        df = df.drop(columns=['vectorized'])
    if df.index.duplicated().any():
        df = df.reset_index()
        df = df.drop(columns = ['id'])
        df = df.rename(columns = {'index': 'id'})
    print(f"Found {len(df)} files")
    return df

# Big corpuses

## Rzepa

In [3]:
corp = 'rzepa'
df_rest, df_eco = data_pipeline(corp)
df_eco = pd.concat([df_eco, eco_selector(df_rest)])
df_eco = validator(df_eco)
df_eco.to_csv(f"eco_{corp}.csv")

{1: 'rzepa', 2: 'gpc', 3: 'newsweek', 4: 'wprost', 5: 'dorzeczy', 6: 'polityka', 7: 'wyborcza', 8: 'wpolityce'}
rzepa
Configuration finished
Dataset of 14082 samples
0    8468
1    5614
Name: label, dtype: int64
Preprocessing ended
Accuracy of model 0.9971611071682044


100%|██████████| 44/44 [01:52<00:00,  2.56s/it]


21981 683
Found 6297 files


## Wyborcza

In [4]:
corp = 'wyborcza'
df_rest, df_eco = data_pipeline(corp)
df_eco = pd.concat([df_eco, eco_selector(df_rest, ease=True)])
df_eco = validator(df_eco)
df_eco.to_csv(f"eco_{corp}.csv")

{1: 'rzepa', 2: 'gpc', 3: 'newsweek', 4: 'wprost', 5: 'dorzeczy', 6: 'polityka', 7: 'wyborcza', 8: 'wpolityce'}
wyborcza
Configuration finished
Dataset of 11694 samples
0    7000
1    4694
Name: label, dtype: int64
Preprocessing ended
Accuracy of model 0.9965811965811966


100%|██████████| 31/31 [01:41<00:00,  3.28s/it]


15314 469
Found 2316 files


## Gazeta Polska Codziennie

In [5]:
corp = 'gpc'
df_rest, df_eco = data_pipeline(corp)
df_eco = pd.concat([df_eco, eco_selector(df_rest, ease=True)])
df_eco = validator(df_eco)
df_eco.to_csv(f"eco_{corp}.csv")

{1: 'rzepa', 2: 'gpc', 3: 'newsweek', 4: 'wprost', 5: 'dorzeczy', 6: 'polityka', 7: 'wyborcza', 8: 'wpolityce'}
gpc
Configuration finished
Dataset of 11696 samples
0    7000
1    4696
Name: label, dtype: int64
Preprocessing ended
Accuracy of model 0.9982905982905983


100%|██████████| 16/16 [00:27<00:00,  1.70s/it]


7531 304
Found 2148 files


## Polityka

In [6]:
corp = 'polityka'
df_rest, df_eco = data_pipeline(corp)
df_eco = pd.concat([df_eco, eco_selector(df_rest, ease=True)])
df_eco = validator(df_eco)
df_eco.to_csv(f"eco_{corp}.csv")

{1: 'rzepa', 2: 'gpc', 3: 'newsweek', 4: 'wprost', 5: 'dorzeczy', 6: 'polityka', 7: 'wyborcza', 8: 'wpolityce'}
polityka
Configuration finished
Dataset of 18921 samples
0    9734
1    9187
Name: label, dtype: int64
Preprocessing ended
Accuracy of model 0.9963021658742737


100%|██████████| 3/3 [00:09<00:00,  3.08s/it]


1046 149
Found 654 files


# Small Corpuses

## Dorzeczy

In [7]:
corp = 'dorzeczy'
df_rest, df_eco = data_pipeline(corp)
df_eco = pd.concat([df_eco, eco_selector(df_rest, ease=True)])
df_eco = validator(df_eco)
df_eco.to_csv(f"eco_{corp}.csv")

{1: 'rzepa', 2: 'gpc', 3: 'newsweek', 4: 'wprost', 5: 'dorzeczy', 6: 'polityka', 7: 'wyborcza', 8: 'wpolityce'}
dorzeczy
Configuration finished
Dataset of 17711 samples
0    8954
1    8757
Name: label, dtype: int64
Preprocessing ended
Accuracy of model 0.9966139954853274


100%|██████████| 2/2 [00:06<00:00,  3.48s/it]


688 41
Found 331 files


## Wprost

In [8]:
corp = 'wprost'
configuration = get_setup(corp)
df_eco, df_non_eco, df_rest = get_current_data(configuration)
df_eco = pd.concat([df_eco, df_rest.drop(index=2260)])
df_eco = validator(df_eco)
df_eco.to_csv(f"eco_{corp}.csv")

{1: 'rzepa', 2: 'gpc', 3: 'newsweek', 4: 'wprost', 5: 'dorzeczy', 6: 'polityka', 7: 'wyborcza', 8: 'wpolityce'}
wprost


/home/ps/Code/Climate/Classification/resources/get_data.py:7: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_eco = pd.read_csv(drive_path / corpus / "eco_result.csv", index_col=['id', 'source'], parse_dates=['date'])
/home/ps/Code/Climate/Classification/resources/get_data.py:8: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_non_eco = pd.read_csv(drive_path / corpus / "non_eco_result.csv", index_col = ['id', 'source'], parse_dates=['date'])


Found 456 files


## Newsweek

In [9]:
corp = 'newsweek'
configuration = get_setup(corp)
df_eco, df_non_eco, df_rest = get_current_data(configuration)
df_all = pd.concat([df_rest['words'].drop(index=2051), df_eco])
df_eco = validator(df_eco)
df_all.to_csv(f"eco_{corp}.csv")

{1: 'rzepa', 2: 'gpc', 3: 'newsweek', 4: 'wprost', 5: 'dorzeczy', 6: 'polityka', 7: 'wyborcza', 8: 'wpolityce'}
newsweek
Found 378 files


## wPolityce

In [10]:
corp = 'wpolityce'
configuration = get_setup(corp)
df_eco, df_non_eco, df_rest = get_current_data(configuration)
df_eco = validator(df_eco)
df_eco.to_csv(f"eco_{corp}.csv")

{1: 'rzepa', 2: 'gpc', 3: 'newsweek', 4: 'wprost', 5: 'dorzeczy', 6: 'polityka', 7: 'wyborcza', 8: 'wpolityce'}
wpolityce
Found 398 files
